# Test run

In [1]:
#using Pkg
#using HDF5
using LDA_CGS
using JSON

┌ Info: Recompiling stale cache file /Users/yanagi/.julia/compiled/v1.0/LDA_CGS/FbSPT.ji for LDA_CGS [8b03fa0e-3994-11e9-179b-21593925b643]
└ @ Base loading.jl:1190


## Get documents

In [2]:
vocab = []
texts = []

open("./data/hep-ph.json", "r") do f
    i=1
    for line in eachline(f)
        jdict = JSON.parse(line)
        words = split(jdict["summary"],r" | |\.|\,|\n|\$")
        push!(texts, words)
        union!(vocab, collect(words))
        i += 1
    end
end

vocab_size,  = size(vocab)
word_to_id = Dict()
id_to_word = Dict()
for i in 1:vocab_size
    word_to_id[vocab[i]] = i
    id_to_word[i] = vocab[i]
end

First make whole corpus

In [3]:
texts_size, = size(texts)
corpus = []
for text in texts
    words_id = map(w->word_to_id[w], text)
    
    counts_d = Dict{Int, Int}()
    for iv in words_id
        counts_d[iv] = get(counts_d, iv, 0) + 1
    end

    counts_d_tuple = []
    for (key, ct) in counts_d
        push!(counts_d_tuple, (key, ct))
    end
    push!(corpus, counts_d_tuple)
end

Then devide it into corpus for training and test.

In [4]:
# devide test and train
train_ratio = 0.8
train_corpus = []
test_corpus = []

for d in corpus
    Nd, = size(d)
    length = Int(round(Nd*0.8))
    push!(train_corpus, d[1:length])
    push!(test_corpus, d[length+1:end])
end

## Run

First make LDA object

In [5]:
K = 20
V = vocab_size
x = LDA(K, V)

LDA(20, 13182, 561156, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, 6.0e-323, 6.4e-323, 7.0e-323, #undef, 25, 1.33e-322, #undef, #undef, #undef)

Then run

In [6]:
burnin = 10
sample = 2
run_LDA(x, train_corpus, test_corpus, burnin, sample)

Burn-in (period=10)...
epoch=1
epoch=2
epoch=3
epoch=4
epoch=5
epoch=6
epoch=7
epoch=8
epoch=9
epoch=10
Sampling from the posterior...
epoch=1, PPL=47.90106467790846
epoch=2, PPL=47.457031128907396


In [8]:
x.Ndk_mean

1000×20 Array{Float64,2}:
  2.0   7.5   0.0   0.0  16.0   0.5  …  0.5   0.5   0.0   4.5   7.5  16.5
 26.0   3.0   2.0   0.0  20.5   0.0     0.0   6.0   4.5   4.5   4.0   2.0
  0.0   1.0   0.0   4.5  50.5   2.5     0.5  12.5   0.0  13.0   4.0   0.5
  2.5   8.0   1.5   6.0  20.5   3.0     8.0  15.5   2.5   1.0   9.0   0.0
  0.0   1.0   0.5   6.0   1.5   1.0     7.0  15.0   3.5   0.0  41.5   1.5
 12.0   0.5   0.5  12.0   6.5   2.5  …  0.0  14.0   8.0   2.5   5.5   2.5
  5.5  26.0   5.5   3.0   0.5   0.5     0.0   2.0   3.0  59.5   1.0   2.0
  3.0   2.5   3.0   1.0   0.0   2.0     0.0   1.0  34.0   0.0   5.5   3.5
  0.0   5.5   2.5   0.0   3.0   5.5     0.0   1.5   3.0   0.0  11.0   5.5
  0.0   4.0   1.5   0.0  18.5   1.5     1.0  25.0   0.5   9.5  42.0   3.5
  0.0  13.0   6.5   4.0   6.5  10.0  …  1.5   9.0   1.0   0.5  21.5   0.0
  3.5   0.5   1.5  15.0   0.0   0.0     2.0  28.5   0.0   0.0   1.5   0.5
 26.5   7.0   0.0   1.0  10.5   0.0     0.0   0.0   1.0  11.5   7.5   2.0
  ⋮         